SETUP AND INITAL CONFIG

In [1]:
from unsloth import FastModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
C:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
C:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1128 11:15:12.972000 5392 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model, tokenizer = FastModel.from_pretrained(
    model_name="google/gemma-3-1b-it",
    max_seq_length=1024,
    load_in_4bit=True,
    load_in_8bit=False,
)

C:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:348: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers=False,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3047
)

Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
from datasets import load_dataset
dataset = load_dataset("madrylab/gsm8k-platinum", "main", split="test")

In [5]:
dataset

Dataset({
    features: ['question', 'answer', 'cleaning_status'],
    num_rows: 1209
})

In [6]:
dataset[0]["question"]

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [7]:
dataset[0]["answer"]


'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'

In [8]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

'18'

In [ ]:
reasoning_start = "<reasoning>"
reasoning_end = "<reasoning"
answer_start = "<answer>"
answer_end = "<answer>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {answer_start}{answer_end}."""


In [10]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})

In [11]:
dataset[0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': '18',
 'cleaning_status': 'consensus',
 'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <reasoning> and <reasoning>.\nThen, provide your solution between <answer><answer>.',
   'role': 'system'},
  {'content': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
   'role': 'user'}]}

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{answer_start}(.+?){answer_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [13]:
match_format.search("<reasoning> Here is my working out<reasoning>"\
                    "<answer>42 <answer>")

<re.Match object; span=(0, 64), match='<reasoning> Here is my working out<reasoning><ans>

In [14]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for output in completions:
        score = 0
        response = output[0]["content"]
        if match_format.search(response) is not None:
            score += 3
            scores.append(score)
        else:
            scores.append(score)
    return scores

In [ ]:
def match_format_partially(completions, **kwargs):
    scores = []
    for output in completions:
        score = 0
        response = output[0]["content"]
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end) == 1 else -0.5
        score += 0.5 if response.count(answer_start) == 1 else -0.5
        score += 0.5 if response.count(answer_end) == 1 else -0.5
        scores.append(score)
    return scores

In [16]:
def check_answer_correctness(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [output[0]["content"] for output in completions]

    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) is not None else None for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        if guess == true_answer:
            score += 3
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            try:
                ratio = float(guess) / float(true_answer)
                if ratio >= 0.9 and ratio <= 1.1:
                    score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2:
                    score += 0.25
                else:
                    score -= 1
            except:
                score -= 1
        scores.append(score)
    return scores

In [17]:
match_numbers = re.compile(
    rf"{answer}.*?([\d\d.]{{1,}})",
    flags= re.MULTILINE | re.DOTALL
)

In [18]:
match_numbers.findall("<answer> The answer is 42.0 <answer>")

['42.0']

In [19]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [output[0]["content"] for output in completions]
    
    extracted_responses = [guess.group(1) if (guess := match_numbers.search(r)) is not None else None for r in responses]

    scores = []
    print("*"*10, f"\nAnswer: {answer[0]}", f"\nExtracted: {extracted_responses[0]}", f"\nComplete Response: {responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        try:
            true_answer = float(true_answer)
            guess = float(guess)
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0.0)
            continue
    return scores


TRAINING 

In [20]:
from trl import GRPOConfig, GRPOTrainer

In [21]:
training_args = GRPOConfig(
    learning_rate= 5e-6,
    adam_beta1= 0.9,
    adam_beta2= 0.99,
    weight_decay= 0.1,
    warmup_ratio= 0.1,
    lr_scheduler_type= "cosine",
    optim= "adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=8,
    max_prompt_length=256,
    max_completion_length=768,
    max_steps=50,
    save_steps=50,
    max_grad_norm=0.1,
    report_to= "wandb",
    output_dir= "outputs"
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [22]:
trainer = GRPOTrainer(
    model= model,
    processing_class= tokenizer,
    reward_funcs= [
        match_format_exactly,
        match_format_partially,
        check_answer_correctness,
        check_numbers
    ],
    args= training_args,
    train_dataset= dataset
)

In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,209 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 13,045,760 of 1,012,931,712 (1.29% trained)
wandb: Currently logged in as: romaizdabeer57 (romaizdabeer57-softverx) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 32768, 'top_p': 0.95}. If this is not desired, please set these values explicitly.


********** 
Answer: 842 
Extracted: None 
Complete Response: <reasoning>
Let $n$ be the number of pupils, which is 29.
Let $c$ be the number of coupons, which is 9.
Each coupon can be redeemed for 100 bottles of apple juice.
Each student receives 2 bottles of apple juice for lunch.
The teacher gives each student 2 bottles of apple juice.

The total number of bottles of apple juice is $9 \times 29 \times 2 = 128 \times 2 = 256$ bottles.
The teacher has 9 coupons.
Each coupon can be redeemed for 100 bottles of apple juice. So the total number of bottles of apple juice the teacher has is $9 \times 100 = 900$ bottles.
The teacher needs to redeem $29 \times 2 = 58$ bottles of apple juice for her own lunch.
The number of bottles of apple juice she has left is $900 - 58 = 842$ bottles.
However, the problem asks how many bottles of apple juice does the teacher have for herself.

Let $x$ be the number of bottles of apple juice the teacher has for herself.
The teacher gives each student 2 bottle

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_partially / mean,rewards / match_format_partially / std,rewards / check_answer_correctness / mean,rewards / check_answer_correctness / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000000,-1.390625,0.391357,448.562500,99.000000,768.000000,0.468750,166.705887,99.000000,394.000000,0,0,0,0,0,0.000731,0.000000,0.000000,-1.812500,0.592289,0.000000,0.000000,0.421875,0.685205
2,0.000000,-0.921875,0.875770,290.093750,138.000000,768.000000,0.031250,274.677399,138.000000,627.000000,No Log,No Log,No Log,No Log,No Log,0.001045,0.000000,0.000000,-1.625000,0.793116,0.000000,0.000000,0.703125,0.760511
3,0.000000,-1.468750,0.393870,530.031250,134.000000,768.000000,0.562500,224.071442,134.000000,760.000000,No Log,No Log,No Log,No Log,No Log,0.000933,0.000000,0.000000,-1.750000,0.672022,0.000000,0.000000,0.281250,0.594837
4,0.000000,-1.531250,0.632812,430.250000,103.000000,768.000000,0.406250,199.157898,103.000000,625.000000,No Log,No Log,No Log,No Log,No Log,0.000848,0.000000,0.000000,-1.812500,0.592289,0.000000,0.000000,0.281250,0.594837
5,0.000000,-0.921875,0.650132,208.750000,54.000000,768.000000,0.062500,171.466675,54.000000,504.000000,No Log,No Log,No Log,No Log,No Log,0.001419,0.000000,0.000000,-1.625000,0.793116,0.000000,0.000000,0.703125,0.760511
6,0.000000,-1.375000,0.503440,375.500000,55.000000,768.000000,0.312500,197.090912,55.000000,546.000000,No Log,No Log,No Log,No Log,No Log,0.001096,0.000000,0.000000,-1.937500,0.353553,0.000000,0.000000,0.562500,0.737804
7,0.000000,-1.656250,0.642388,406.781250,35.000000,768.000000,0.375000,190.050003,35.000000,593.000000,No Log,No Log,No Log,No Log,No Log,0.002088,0.000000,0.000000,-1.937500,0.353553,0.000000,0.000000,0.281250,0.594837
8,0.000000,-1.265625,0.622812,491.218750,123.000000,768.000000,0.437500,275.944458,123.000000,722.000000,No Log,No Log,No Log,No Log,No Log,0.002240,0.000000,0.000000,-1.687500,0.737804,0.000000,0.000000,0.421875,0.685205
9,0.000000,-2.000000,0.000000,714.187500,381.000000,768.000000,0.750000,552.750000,381.000000,753.000000,No Log,No Log,No Log,No Log,No Log,0.004805,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000


********** 
Answer: 34 
Extracted: 34.00 
Complete Response: <reasoning>
Let's calculate the earnings from each type of root crop.

- Yams: 6 yams * $1.5/yam = $9.00
- Sweet potatoes: 10 sweet potatoes * $2/sweet potato = $20.00
- Carrots: 4 carrots * $1.25/carrot = $5.00

Total earnings = $9.00 + $20.00 + $5.00 = $34.00

Therefore, Jen will earn $34.00.

<answer>
$34.00
********** 
Answer: 10 
Extracted: None 
Complete Response: <reasoning>
Let the number of dishwasher pods in the regular box be $n = 100$. The cost of the regular box is $12.
The special size box has 20% more pods than the regular box, so the number of pods in the special box is $n + 0.20n = 1.2n = 1.2(100) = 120$ pods.
The cost of the special size box is $12 but with a discount of 20%, so the cost is $12 - 0.20(12) = 12 - 2.4 = 9.6$.
We want to find out how many cycles we can run with $1 if we use 1 pod per cycle.
The number of pods in the special box is 120.
We want to find the number of cycles we can run with $1.
Th

In [ ]:
model.save_pretrained("gemma-3-1b-finetuned")  # Local saving
tokenizer.save_pretrained("gemma-3-1b-finetuned")  # Local saving
model.push_to_hub("Romaiz_Dabeer/gemma-3-1b-finetuned-unsloth") # Online saving
tokenizer.push_to_hub("Romaiz_Dabeer/gemma-3") # Online saving

In [ ]:
model.save_pretrained_merged("gemma-3-1b-finetuned-unsloth", tokenizer)

In [ ]:
model.push_to_hub_merged(
        "Romaiz_Dabeer/gemma-3-finetune-finetuned-unsloth", tokenizer
    )

In [ ]:
model.push_to_hub_gguf(
        "Romaiz_Dabeer/gemma-3-1b-finetuned-BF16_gguf",
        tokenizer,
        quantization_method = "BF16", # Only Q8_0, BF16, F16 supported
    )

In [ ]:
torch.cuda.empty_cache()